In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("data/clean_mono_norm.csv")
senti = pd.read_csv("data/sentiments.csv")
ftprobs = pd.read_csv("data/cleanlabels_ftprobs.csv")

In [ ]:
df["ft_bug"] = ftprobs["__label__bug"] 
df["ft_feature"] = ftprobs["__label__feature"] 
df["ft_other"] = ftprobs["__label__other"] 

In [ ]:
df["title_sentistrenght_p"] = senti["title_sentistrenght"].apply(lambda x: x.split(',')[0])
df["title_sentistrenght_n"] = senti["title_sentistrenght"].apply(lambda x: x.split(',')[1])

df["body_sentistrenght_p"] = senti["body_sentistrenght"].apply(lambda x: x.split(',')[0])
df["body_sentistrenght_n"] = senti["body_sentistrenght"].apply(lambda x: x.split(',')[1])

df["title_polarity"] = senti["title_textblob"].apply(lambda x: x.split(',')[0])
df["title_subjectivity"] = senti["title_textblob"].apply(lambda x: x.split(',')[1])

df["body_polarity"] = senti["body_textblob"].apply(lambda x: x.split(',')[0])
df["body_subjectivity"] = senti["body_textblob"].apply(lambda x: x.split(',')[1])

In [ ]:
df["positive_title_sentistrenght_n"] = df["title_sentistrenght_n"].astype(int).abs()
df["positive_body_sentistrenght_n"] = df["body_sentistrenght_n"].astype(int).abs()
df["positive_title_polarity"] = df["title_polarity"].astype(float) + 1
df["positive_body_polarity"] = df["body_polarity"].astype(float) + 1
df['num_of_qmark'] = df['num_of_qmark'].fillna(0)

In [ ]:
df.to_csv("data/final_dataset.csv", index=False)

In [ ]:
df_text_columns = ['number', 'repository_url', 'title', 'body', 'title_processed', 'body_processed', 
                   'title_proc_lem', 'body_proc_lem', 'label_cat', 'test_tag']

df[df_text_columns].to_csv("data/finaldf_text_columns.csv", index = False)

In [ ]:
df.drop(columns = ['title', 'body', 'title_processed', 'body_processed', 
                   'title_proc_lem', 'body_proc_lem'], inplace=True)

df.to_csv("data/finaldf_nontext_columns.csv", index = False)

#### normalization

In [2]:
# df = pd.read_csv("data/finaldf_nontext_columns.csv")

In [ ]:
train = df[df.test_tag==0]
test = df[df.test_tag==1]

unnormalized_columns = ['comments', 'num_of_assignees',
                        'reaction_total_count', 'reaction_+1', 'reaction_-1', 'reaction_laugh', 'reaction_hooray', 'reaction_confused', 'reaction_heart', 'reaction_rocket', 'reaction_eyes', 
                        'title_len', 'body_len', 'title_words_num', 'body_words_num', 'title_alpha_len', 'body_alpha_len', 
                        'body_processed_len', 'title_processed_len', 'title_processed_words_num', 'body_processed_words_num',
                        'title_lem_len', 'title_lem_words_num', 'body_lem_len', 'body_lem_words_num',
                        'num_of_sharps', 'num_of_at', 'num_of_qmark', 'num_of_codesnippets', 'num_of_functions', 'num_of_issues', 'num_of_paths', 'num_of_dates', 'num_of_times', 'num_of_urls', 'num_of_emails', 'num_of_obligations',
                        'title_sentistrenght_p', 'positive_title_sentistrenght_n', 'body_sentistrenght_p', 'positive_body_sentistrenght_n',
                        'positive_title_polarity', 'positive_body_polarity']

min_max_scaler = MinMaxScaler()
train[unnormalized_columns]  = min_max_scaler.fit_transform(train[unnormalized_columns])
test[unnormalized_columns]  = min_max_scaler.transform(test[unnormalized_columns])

norm_df = pd.concat([train, test], ignore_index=True)
norm_df.to_csv("data/normdf_nontext_columns.csv", index = False)